- A graph is a mathematical construct where we have a set of object (called nodes) in which some pairs are related.

- "We call each of the nodes a _vertex_ and each of the connections an _edge_.

- Graphs help us to think abstractly about a problem.

# Building a graph framework

We will build an edge class that is flexible.

Note: The __Edge__ class uses a new feature in Python 3.7: dataclasses. A class marked with the decorator @dataclass saves us the trouble of creating an __init()__ method that instantiates instance variables for any variables declared with type annotations in the class's body.

In [1]:
from __future__ import annotations
from dataclasses import dataclass

@dataclass
class Edge:
    u: int # the 'from' vertex
    v: int # the 'to' vertex
    
    # the reversed method is meant to return an Edge that travels the opposite direction
    def reversed(self) -> Edge:
        return Edge(self.v, self.u)
    
    def __str__(self) -> str:
        return f'{self.u} -> {self.u}'

## Graph class

In [2]:
from typing import TypeVar, Generic, List, Optional

V = TypeVar('V') # type of the vertices in the graph

# We initialise a list of vertices with edges to be added later
class Graph(Generic[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[Edge]] = [[] for _ in vertices] #initialised to a list of blank lists
        
    @property
    def vertex_count(self) -> int:
        return len(self._vertices) # number of vertices

    @property
    def edge_count(self) -> int:
        return sum(map(len, self._edges))

    # add a vertex to the graph and return its index
    def add_vertex(self, vertex: V)-> int:
        self._vertices.append(vertex)
        self._edges.append([]) # add empty list for containing edges
        return self.vertex_count - 1 # return index of added vertex

    # This is an undirected graph,
    # so we always add edges in both directions
    def add_edge(self, edge: Edge) -> None:
        self._edges[edge.u].append(edge)
        self._edges[edge.v].append(edge.reversed())
    
    # Add an edge using vertex indices (convenience method)
    def add_edge_by_indices(self, u: int, v: int) -> None:
        edge: Edge = Edge(u, v)
        self.add_edge(edge)
    
    # Add an edge by looking up (existing) vertex indices (convenience method)
    def add_edge_by_vertices(self, first: V, second: V) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v)
    
    # Find the vertex at a specific index
    def vertex_at(self, index: int) -> V:
        return self._vertices[index]
    
    # Find the index of a vertex in the graph
    def index_of(self, vertex: V) -> int:
        return self._vertices.index(vertex)

    # Find the vertices that a vertex t some index is connected to:
    def neighbors_for_index(self, index: int) -> List[V]:
        return list(map(self.vertex_at, [e.v for e in self._edges[index]]))

    # Look up a vertice's index and find its neighbours (convience method)
    def neighbors_for_vertex(self, vertex: V) -> List[V]:
        return self.neighbors_for_index(self.index_of(vertex))

    # Return all of the edges associated with a vertex at some index
    def edges_for_vertex(self, vertex: V) -> List[Edge]:
        return self.edges_for_index(self.index_of(vertex))

    # make sure it is easy to pretty-print a Graph
    def __str__(self) -> str:
        desc: str = ''
        for i in range(self.vertex_count):
            desc += f'{self.vertex_at(i)} -> {self.neighbors_for_index(i)} \n'
        return desc

# Working with Edge and Graph (4.2.1)

In [3]:
# test basic Graph construction
city_graph: Graph[str] = Graph(['Seattle', 'San Francisco', 'Los Angeles', 
                                'Riverside', 'Phoenix', 'Chicago', 'Boston',
                                'New York','Atlanta', 'Miami', 'Dallas', 
                                'Houston', 'Detroit', 'Philadelphia',
                                'Washington'])
city_graph.add_edge_by_vertices('Seattle', 'Chicago')
city_graph.add_edge_by_vertices('Seattle', 'San Francisco')
city_graph.add_edge_by_vertices('San Francisco', 'Riverside')
city_graph.add_edge_by_vertices('San Francisco', 'Los Angeles')
city_graph.add_edge_by_vertices('Los Angeles', 'Riverside')
city_graph.add_edge_by_vertices('Los Angeles', 'Phoenix')
city_graph.add_edge_by_vertices('Riverside', 'Phoenix')
city_graph.add_edge_by_vertices('Riverside', 'Chicago')
city_graph.add_edge_by_vertices('Phoenix', 'Dallas')
city_graph.add_edge_by_vertices('Phoenix', 'Houston')
city_graph.add_edge_by_vertices('Dallas', 'Chicago')
city_graph.add_edge_by_vertices('Dallas', 'Atlanta')
city_graph.add_edge_by_vertices('Dallas', 'Houston')
city_graph.add_edge_by_vertices('Houston', 'Atlanta')
city_graph.add_edge_by_vertices('Houston', 'Miami')
city_graph.add_edge_by_vertices('Atlanta', 'Chicago')
city_graph.add_edge_by_vertices('Atlanta', 'Washington')
city_graph.add_edge_by_vertices('Atlanta', 'Miami')
city_graph.add_edge_by_vertices('Miami', 'Washington')
city_graph.add_edge_by_vertices('Chicago', 'Detroit')
city_graph.add_edge_by_vertices('Detroit', 'Boston')
city_graph.add_edge_by_vertices('Detroit', 'Washington')
city_graph.add_edge_by_vertices('Detroit', 'New York')
city_graph.add_edge_by_vertices('Boston', 'New York')
city_graph.add_edge_by_vertices('New York', 'Philadelphia')
city_graph.add_edge_by_vertices('Philadelphia', 'Washington')
print(city_graph)

# Revisiting Breadth-first search (BFS)

“What is the shortest path between Boston and Miami?”

We can reuse the generic code in Chapter 2.

Note: https://stackoverflow.com/questions/4383571/importing-files-from-different-folder

In [4]:
import sys
sys.path.append('../Chapter 2/')
from generic_search import bfs, Node, node_to_path

# bfs takes 3 arguments: starting point, a callable that returns a boolean value, and a callable that returns
# a list of lists
bfs_result: Optional[Node[V]] = bfs('Boston', lambda x: x == 'Miami', city_graph.neighbors_for_vertex)

if bfs_result is None:
    print('No solution found using breadth first search!')
else:
    path: List[V] = node_to_path(bfs_result)
    print('Path from Boston to Miami \n')
    print(path)

Path from Boston to Miami 

['Boston', 'Detroit', 'Washington', 'Miami']


# Minimising the cost of building the network

In the previous breadth first search, we looked at the path with the fewest steps. However, in practice, edges have weights representing some kind of cost of travelling from one vertex to another. In the current example, we will add weights that represent the distances in miles between the cities (vertices).

To handle weights, we need a subclass of __Edge__ - __WeightedEdge__